# Testing - Agents
## Introduction

## Installation

Note : we are pinning the inspect_ai due to a recent breaking change

In [2]:
%pip install -q openai anthropic ipywidgets colorama
import os
os.environ['XDG_RUNTIME_DIR']="/tmp"
os.environ['INSPECT_EVAL_MODEL'] = "openai/gpt-4o-mini"

from helpers.reporter.pretty import pretty_results


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Run a shell one with tools

We add all the models we want to test across

In [ ]:
from inspect_ai.agent import Agent, AgentState, agent, as_solver, as_tool
from inspect_ai.model import ChatMessageSystem, get_model
from inspect_ai.tool import web_browser

@agent
def web_surfer() -> Agent:
    async def execute(state: AgentState) -> AgentState:
        """Web research assistant."""
      
        # some general guidance for the agent
        state.messages.append(
            ChatMessageSystem(
                content="You are an expert at using a " + 
                "web browser to answer questions."
            )
        )

        # run a tool loop w/ the web_browser 
        messages, output = await get_model().generate_loop(
            state.messages, tools=web_browser(interactive=False),
        )

        # update and return state
        state.output = output
        state.messages.extend(messages)
        return state

    return execute

from inspect_ai.agent import react
from inspect_ai.tool import web_browser


from inspect_ai.solver import generate, use_tools
from inspect_ai import Task, eval, task
from inspect_ai.dataset import json_dataset
from inspect_ai.scorer import includes
from inspect_ai.dataset import Sample

@task
def custom_agent():

    dataset = [
     Sample(input="Where does patrick debois live ? Use wikipedia", target="Belgium")
    ]
 
    task = Task(
        dataset=dataset,
        solver=web_surfer(),
        scorer=includes(),
        sandbox="docker"
    )
    return task

results = eval(custom_agent,log_level="info",display="none")
print(pretty_results(results))

Output()


Status: success Model: openai/gpt-4o-mini
/workspaces/workshop-testing/lessons/04-testing/logs/2025-05-07T13-13-35+00-00_myagent_dEhDptg8DrtjJ5iCfqcaiG.eval
======= Sample ========
input : Where does patrick debois live ? Use wikipedia
target: Belgium
 user       > Where does patrick debois live ? Use wikipedia
 system     > You are an expert at using a web browser to answer questions.
 assistant [tool:web_browser_go] > {'url': 'https://en.wikipedia.org/wiki/Patrick_Debois'}
 assistant  > 
 tool[web_browser_go] +> main content:
Patrick Debois Patrick Debois This article is only available in this language. Add the article for other languages Namespaces Page tools Tools Appearance Appearance hide hide Patrick Debois From Wikipedia, the free encyclopedia  Look for Patrick Debois on one of Wikipedia's sister projects sister projects : Wiktionary Wiktionary (dictionary) Wikibooks Wikibooks (textbooks) Wikiquote Wikiquote (quotations) Wikisource Wikisource (library) Wikiversity Wikiversity 

## Handoff

In [4]:
from inspect_ai.agent import react
from inspect_ai.tool import web_browser
from inspect_ai.solver import generate, use_tools
from inspect_ai import Task, eval, task
from inspect_ai.dataset import json_dataset
from inspect_ai.scorer import includes
from inspect_ai.dataset import Sample


web_surfer = react(
    name="web_surfer",
    description="Web research assistant",
    prompt="You are a tenacious web researcher that is expert "
           + "at using a web browser to answer questions.",
    tools=web_browser()   
)

from inspect_ai.agent import handoff
from inspect_ai.dataset import Sample

supervisor = react(
    prompt="You are an agent that can answer addition " 
            + "problems and do web research.",
    tools=[ handoff(web_surfer)]
)

agent_handoff = Task(
    dataset=[
        Sample(input="Please add 1+1 then tell me what " 
                     + "movies were popular in 2020")
    ],
    solver=supervisor,
    sandbox="docker",
)

results = eval(agent_handoff, log_level="info", display="none")
print(pretty_results(results))

Output()


Status: success Model: openai/gpt-4o-mini
/workspaces/workshop-testing/lessons/04-testing/logs/2025-05-07T13-14-03+00-00_task_VTDVML5oR4N9H8iJuZZuST.eval
======= Sample ========
input : Please add 1+1 then tell me what movies were popular in 2020
target: 
 system     > You are an agent that can answer addition problems and do web research.


You are part of a multi-agent system designed to make agent coordination and
execution easy. Agents uses two primary abstraction: **Agents** and **Handoffs**.
An agent encompasses instructions and tools and can hand off a conversation to
another agent when appropriate. Handoffs are achieved by calling a handoff function,
generally named `transfer_to_<agent_name>`. Transfers between agents are handled
seamlessly in the background; do not mention or draw attention to these transfers
in your conversation with the user.



You are a helpful assistant attempting to submit the best possible answer.
You have several tools available to help with finding t